연	월	일	시	건물번호	기온(C)	강수량(mm)	풍속(m/s)	습도(%)
 - XGBRegressor: 0.9655916940702893
 - Dacon 점수: 17.44154

넣을 수 있는 column 다 넣은 경우
 - XGBRegressor: 0.9737195720193215
 - Dacon 점수: 16.55132

 - '습도(%)', 'ESS저장용량(kWh)', '연', '풍속(m/s)', '강수량(mm)' 제거
 - XGBRegressor: 0.9747093891268452
 - Dacon 점수: 16.55132
 - RanForestRegressor: 0.9866593482821421
 - Dacon 점수: 11.80555

In [1]:
import numpy as np

def smape(true, predict):  # Symmetric Mean Absolute Percentage Error
    return 100/len(true) * np.sum(2 * np.abs(true - predict) / (np.abs(true) + np.abs(predict)))

In [2]:
import pandas as pd


building_info = pd.read_csv('./data/building_info.csv')
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')
sample = pd.read_csv('./data/sample_submission.csv')
# display(train.head(1))
# display(test.head(1))
# display(building_info.head(1))

In [3]:
from sklearn.preprocessing import OneHotEncoder

building_info.replace('-', 0, inplace=True)
onehotencoder = OneHotEncoder(sparse_output=False)
building_type = onehotencoder.fit_transform(building_info[['건물유형']])

temp = pd.DataFrame(building_type)
temp.columns = onehotencoder.categories_[0]


building = pd.concat([building_info, temp], axis=1)
del building['건물유형']
building = building.astype('float')
# building.head(3)

In [4]:
def data_preprocessing(data):  
    data.fillna(0, inplace=True)
    data['연'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][:4]))
    data['월'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][4:6]))
    data['일'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][6:8]))
    data['시'] = data['num_date_time'].apply(lambda x: int(x.split(' ')[1]))
    del data['일시']
    del data['num_date_time']
    
    data = pd.merge(data, building, on='건물번호')
    del data['건물번호']
    # display(data.head(3))
    return data

train.drop(['일조(hr)', '일사(MJ/m2)'], axis=1, inplace=True)
train = data_preprocessing(train)

In [6]:
y = train['전력소비량(kWh)']
del train['전력소비량(kWh)']

In [7]:
# 변수 선정
from itertools import *
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

origin_train = train.copy()
print(origin_train.columns)
origin_train.drop(['습도(%)', 'ESS저장용량(kWh)', '연', '풍속(m/s)', '강수량(mm)'], axis=1, inplace=True)
print()
print(origin_train.columns)
candidate = []

result = []

for i in tqdm(range(len(origin_train.columns) - 12)):
    col = list(origin_train.columns[:i])
    col.extend(origin_train.columns[i+1:])
    X = origin_train[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=987)

    XR = XGBRegressor()
    XR.fit(X_train, y_train)
    y_pred = XR.predict(X_test)
    score = smape(y_test, y_pred)
    result.append((origin_train.columns[i], score))

result.sort(key=lambda x: x[1], reverse=True)
for i in result:
    print(i)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

columns = ['기온(C)', '월', '일', '시', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'PCS용량(kW)', '건물기타', '공공',
           '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트', '호텔및리조트']

data = train[columns]

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.5, random_state=987)

In [9]:
from xgboost import XGBRegressor

XR = XGBRegressor()
XR.fit(X_train, y_train)
y_pred = XR.predict(X_test)
smape(y_test, y_pred)

12.050286769802911

In [15]:
from sklearn.ensemble import RandomForestRegressor

RFR = RandomForestRegressor()
RFR.fit(X_train, y_train)
y_pred = RFR.predict(X_test)
smape(y_test, y_pred)

6.825222554179898

In [11]:
from lightgbm import LGBMRegressor

LR = LGBMRegressor()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
smape(y_test, y_pred)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 566
[LightGBM] [Info] Number of data points in the train set: 102000, number of used features: 20
[LightGBM] [Info] Start training from score 2455.908183


14.800485313114013

In [16]:
import datetime

test = pd.read_csv('./data/test.csv')
test_data = data_preprocessing(test)
test_data = test_data[columns]
answer = RFR.predict(test_data)

submission = pd.DataFrame()
submission['num_date_time'] = sample['num_date_time']
submission['answer'] = answer

now = datetime.datetime.now().strftime("%Y-%m-%d %H시%M분%S초")
submission.to_csv(f'{now}.csv', index=False)